# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121273e+02     1.448108e+00
 * time: 0.684650182723999
     1     1.081690e+01     7.917203e-01
 * time: 1.9819681644439697
     2    -1.174167e+01     9.952398e-01
 * time: 2.623244047164917
     3    -3.411153e+01     7.034351e-01
 * time: 3.598903179168701
     4    -4.769210e+01     5.228962e-01
 * time: 4.53157901763916
     5    -5.701983e+01     2.020207e-01
 * time: 5.48606014251709
     6    -5.983951e+01     1.203731e-01
 * time: 6.153020143508911
     7    -6.088605e+01     4.802719e-02
 * time: 6.775328159332275
     8    -6.128136e+01     6.474652e-02
 * time: 7.413881063461304
     9    -6.157489e+01     4.293037e-02
 * time: 8.050973176956177
    10    -6.179167e+01     3.347803e-02
 * time: 8.703809022903442
    11    -6.196198e+01     2.045225e-02
 * time: 9.33138918876648
    12    -6.202657e+01     1.901611e-02
 * time: 9.960750102996826
    13    -6.209191e+01     1.198616e-02
 * time: 10.590025186538696
    14  

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671077
    AtomicLocal         -18.8557718
    AtomicNonlocal      14.8522664
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485391 
    Xc                  -19.3336828

    total               -62.261666460865